<a href="https://colab.research.google.com/github/MisaTecMNA/PSAC_MNA26/blob/main/A01796906_A4.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 4.2 Ejercicio de programación 1
### Pruebas de software y aseguramiento de la calidad
### MISAEL LOPEZ SANCHEZ - A01796906

Comando de instalación de PyLint, muestra de ejecución desde google colab.

In [1]:
pip install pylint

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.4/536.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 3.2 MB/s eta 0:00:00


Verificar la versión de `pylint` ejecutando:

In [2]:
!pylint --version

pylint 4.0.4
astroid 4.0.3
Python 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]


#4.2 Ejercicios de programación 1 Ejercicio 1 - Compute statistics

In [3]:
import os
from google.colab import drive

# 1. Montar Google Drive
drive.mount('/content/drive')

# 2. Ruta donde se encuentran los archivos (Mi Tec google drive)
ruta_insumos = "/content/drive/MyDrive/PruebasSoftware2026/A4.2_ArchivosApoyo/P1"

# 3. Cambiar el directorio de trabajo
if os.path.exists(ruta_insumos):
    os.chdir(ruta_insumos)
    print(f"Directorio cambiado exitosamente a: {os.getcwd()}")
    print("Archivos en esta carpeta:")
    # Listamos para verificar que ves TC1.txt, TC2.txt, etc.
    print(os.listdir())
else:
    print(f"La ruta {ruta_insumos} no existe. Verificar.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Directorio cambiado exitosamente a: /content/drive/MyDrive/PruebasSoftware2026/A4.2_ArchivosApoyo/P1
Archivos en esta carpeta:
['TC5.txt', 'TC7.txt', 'TC4.txt', 'TC1.txt', '.DS_Store', 'TC3.txt', 'A4.2.P1.Results-errata.txt', 'TC2.txt', 'TC6.txt', 'computeStatistics.py', 'StatisticsResults.txt']


## Programa compute statistics

Se ejecuta el comando `writefile` que permite escribir y un archivo `.py` en la ruta seleccionada.

In [4]:
%%writefile computeStatistics.py
# pylint: disable=invalid-name
"""
Programa: computeStatistics.py
Descripción: Calcula estadísticas descriptivas de múltiples archivos de texto.
Salida: Archivo .txt con formato tabular.
"""
import sys
import time


def read_file(filename):
    """
    Lee un archivo y retorna una lista de números.
    Maneja datos inválidos e imprime errores en consola de error (stderr).
    """
    data = []
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            for line_num, line in enumerate(file, 1):
                clean_line = line.strip()
                if not clean_line:
                    continue
                try:
                    number = float(clean_line)
                    data.append(number)
                except ValueError:
                    # Reportar error sin detener ejecución
                    sys.stderr.write(f"Error en {filename}, línea {line_num}: "
                                     f"'{clean_line}' no es numérico.\n")
    except FileNotFoundError:
        sys.stderr.write(f"Error: El archivo '{filename}' no fue encontrado.\n")
        return None
    return data


def calculate_stats(data):
    """Calcula métricas: Count, Mean, Median, Mode, SD, Variance."""
    if not data:
        return None

    count = len(data)
    mean = sum(data) / count

    # Mediana
    sorted_data = sorted(data)
    mid = count // 2
    if count % 2 == 1:
        median = sorted_data[mid]
    else:
        median = (sorted_data[mid - 1] + sorted_data[mid]) / 2.0

    # Moda
    frequency = {}
    for item in data:
        frequency[item] = frequency.get(item, 0) + 1

    max_freq = max(frequency.values())
    # Si todos los números aparecen 1 sola vez, no hay moda útil (ej. floats)
    if max_freq == 1:
        mode = "#N/A"
    else:
        modes = [k for k, v in frequency.items() if v == max_freq]
        mode = modes[0]  # Tomamos el primero si hay múltiples

    # Varianza y Desviación Estándar
    if count < 2:
        variance = 0.0
    else:
        variance = sum((x - mean) ** 2 for x in data) / (count - 1)

    std_dev = variance ** 0.5

    return {
        "COUNT": count, "MEAN": mean, "MEDIAN": median,
        "MODE": mode, "SD": std_dev, "VARIANCE": variance
    }


def print_results(results, filenames, elapsed_time):
    """
    Imprime los resultados en formato tabular (matriz transpuesta)
    y los guarda en StatisticsResults.txt.
    """
    metrics = ["COUNT", "MEAN", "MEDIAN", "MODE", "SD", "VARIANCE"]

    # Construcción de encabezados
    # Usamos tabuladores \t para alinear columnas
    header = "TC\t" + "\t".join([name.replace(".txt", "") for name in filenames])

    rows = [header]

    for metric in metrics:
        row_parts = [metric]
        for name in filenames:
            val = results[name][metric]

            # Formateo de salida similar al ejemplo
            if val == "#N/A":
                row_parts.append(val)
            elif metric == "COUNT":
                row_parts.append(f"{int(val)}")
            elif metric == "MODE":
                row_parts.append(f"{val}")
            else:
                # Flotantes con 2 decimales o formato general si es muy grande
                row_parts.append(f"{val:.2f}")

        rows.append("\t".join(row_parts))

    final_output = "\n".join(rows)

    # 1. Imprimir en pantalla
    print(final_output)
    print(f"\nTiempo de ejecución: {elapsed_time:.6f} s")

    # 2. Guardar en archivo .txt
    output_filename = "StatisticsResults.txt"
    with open(output_filename, "w", encoding='utf-8') as f:
        f.write(final_output)
        f.write(f"\n\nTiempo de ejecución: {elapsed_time:.6f} s")

    print(f"\nArchivo generado exitosamente: {output_filename}")


def main():
    """Función principal."""
    if len(sys.argv) < 2:
        print("Uso: python computeStatistics.py TC1.txt TC2.txt ...")
        sys.exit(1)

    filenames = sys.argv[1:]
    start_time = time.time()

    all_results = {}

    # Procesar cada archivo
    for filename in filenames:
        data = read_file(filename)
        if data is None:
            # Si no se lee, llenamos con ceros para mantener la tabla
            all_results[filename] = {k: 0 for k in ["COUNT", "MEAN", "MEDIAN",
                                                    "MODE", "SD", "VARIANCE"]}
            continue

        stats = calculate_stats(data)
        all_results[filename] = stats

    end_time = time.time()
    elapsed = end_time - start_time

    print_results(all_results, filenames, elapsed)


if __name__ == "__main__":
    main()

Overwriting computeStatistics.py


Ejecutamos el comando `pylint` sobre el archivo *computestatistics.py* y ya no obtenemos errores. El código esta completo y cumple los estandares.

In [5]:
# 2. Ejecutar la auditoría sobre tu archivo
!pylint computeStatistics.py


------------------------------------
Your code has been rated at 10.00/10



Ejecución del programa y resultados en consola.

In [6]:
!python computeStatistics.py TC1.txt TC2.txt TC3.txt TC4.txt TC5.txt TC6.txt TC7.txt

Error en TC1.txt, línea 400: '405s' no es numérico.
Error en TC5.txt, línea 5: 'ABA' no es numérico.
Error en TC5.txt, línea 155: '23,45' no es numérico.
Error en TC5.txt, línea 232: '11;54' no es numérico.
Error en TC5.txt, línea 239: 'll' no es numérico.
Error en TC7.txt, línea 183: 'ABBA' no es numérico.
Error en TC7.txt, línea 229: 'ERROR' no es numérico.
TC	TC1	TC2	TC3	TC4	TC5	TC6	TC7
COUNT	399	1977	12624	12624	307	3000	12767
MEAN	241.91	250.78	249.78	149.00	241.50	187906599279774433280.00	247467395499716247552.00
MEDIAN	239.00	247.00	249.00	147.75	241.00	188008049965542998016.00	246640973074290016256.00
MODE	393.0	230.0	94.0	123.75	393.0	#N/A	#N/A
SD	145.39	144.21	145.32	130.42	145.70	107399951657667608576.00	144611310601233285120.00
VARIANCE	21139.29	20795.89	21118.95	17009.27	21229.17	11534749616069340657613597980597774450688.00	20912431153806367046598590134856523448320.00

Tiempo de ejecución: 1.565074 s

Archivo generado exitosamente: StatisticsResults.txt


#4.2 Ejercicios de programación 1 Ejercicio 2 - Converter


In [7]:
import os
import sys
from google.colab import drive

# 1. Montar Drive
if not os.path.ismount('/content/drive'):
    drive.mount('/content/drive')

# 2. Moverse a la carpeta del Programa 2 (Ubicado en mi TEC google drive)
ruta_p2 = "/content/drive/MyDrive/PruebasSoftware2026/A4.2_ArchivosApoyo/P2"

if os.path.exists(ruta_p2):
    os.chdir(ruta_p2)
    print(f"Directorio de trabajo: {os.getcwd()}")
    print("Archivos disponibles:", os.listdir())
else:
    print(f"Error: La ruta {ruta_p2} no existe. Verificar.")

Directorio de trabajo: /content/drive/MyDrive/PruebasSoftware2026/A4.2_ArchivosApoyo/P2
Archivos disponibles: ['A4.2.P2.Results.txt', 'TC3.txt', 'TC4.txt', 'TC2.txt', 'TC1.txt', 'convertNumbers.py', 'ConvertionResults.txt']


## Programa converterNumbers

Se genera el programa `converterNumbers.py` que contiene la lógica para convertir números a binario y hexadecimal. El archivo es creado en la carpeta del problema 2 `P2`

In [8]:
%%writefile convertNumbers.py
"""
Programa: convertNumbers.py
Descripción: Convierte números de archivos a binario y hexadecimal.
Salida: Archivo ConvertionResults.txt con formato tabular.
"""
import sys
import time


def to_binary(number):
    """
    Convierte entero a binario usando algoritmo de división sucesiva.
    Maneja negativos con signo simple.
    """
    if number == 0:
        return "0"

    is_negative = number < 0
    num = abs(number)
    binary = ""

    while num > 0:
        remainder = num % 2
        binary = str(remainder) + binary
        num = num // 2

    if is_negative:
        return "-" + binary
    return binary


def to_hexadecimal(number):
    """
    Convierte entero a hexadecimal usando algoritmo de residuos.
    """
    if number == 0:
        return "0"

    hex_map = "0123456789ABCDEF"
    is_negative = number < 0
    num = abs(number)
    hex_str = ""

    while num > 0:
        remainder = num % 16
        hex_str = hex_map[remainder] + hex_str
        num = num // 16

    if is_negative:
        return "-" + hex_str
    return hex_str


def process_file(filename):
    """
    Lee un archivo y retorna una lista de tuplas (item, numero, bin, hex).
    """
    results = []
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f, 1):
                text = line.strip()
                if not text:
                    continue
                try:
                    num = int(text)
                    bin_val = to_binary(num)
                    hex_val = to_hexadecimal(num)
                    results.append((i, num, bin_val, hex_val))
                except ValueError:
                    sys.stderr.write(f"Error {filename} linea {i}: "
                                     f"'{text}' invalido.\n")
                    results.append((i, text, "#N/A", "#N/A"))
    except FileNotFoundError:
        sys.stderr.write(f"Error: Archivo '{filename}' no encontrado.\n")
        return None
    return results


def print_results(all_results, elapsed_time):
    """
    Imprime resultados concatenados y guarda en archivo.
    """
    lines = []

    for filename, data in all_results.items():
        if data is None:
            continue

        # Nombre de columna dinámico basado en el archivo (TC1, TC2...)
        col_name = filename.replace(".txt", "")

        # Encabezado para este archivo
        header = f"ITEM\t{col_name}\tBIN\tHEX"
        lines.append(header)

        for item, num, bin_v, hex_v in data:
            # Formato de fila
            row = f"{item}\t{num}\t{bin_v}\t{hex_v}"
            lines.append(row)

        # Espacio entre archivos
        lines.append("")

    final_output = "\n".join(lines).strip()

    # Pie de página con tiempo
    time_info = f"\n\nTiempo de ejecución: {elapsed_time:.6f} s"

    # 1. Imprimir en pantalla
    print(final_output)
    print(time_info)

    # 2. Guardar en archivo
    with open("ConvertionResults.txt", "w", encoding='utf-8') as f:
        f.write(final_output)
        f.write(time_info)

    print("\nArchivo 'ConvertionResults.txt' generado exitosamente.")


def main():
    """Función principal."""
    if len(sys.argv) < 2:
        print("Uso: python convertNumbers.py TC1.txt TC2.txt ...")
        sys.exit(1)

    start_time = time.time()
    filenames = sys.argv[1:]
    all_data = {}

    for fname in filenames:
        all_data[fname] = process_file(fname)

    end_time = time.time()
    elapsed = end_time - start_time

    print_results(all_data, elapsed)


if __name__ == "__main__":
    main()

Overwriting convertNumbers.py


Se aplica el comando `pylint` sobre el archivo *convertNumbers.py* y se observa que ya no se producen errores. El código esta completo y no presenta problemas.

In [9]:
!pylint convertNumbers.py

************* Module convertNumbers
convertNumbers.py:1:0: C0103: Module name "convertNumbers" doesn't conform to snake_case naming style (invalid-name)

-----------------------------------
Your code has been rated at 9.88/10



Ejecución del programa y resultados en consola.

In [10]:
!python convertNumbers.py TC1.txt TC2.txt TC3.txt TC4.txt

Error TC4.txt linea 8: 'ABC' invalido.
Error TC4.txt linea 21: 'ERR' invalido.
Error TC4.txt linea 41: 'VAL' invalido.
ITEM	TC1	BIN	HEX
1	6980368	11010101000001100010000	6A8310
2	5517055	10101000010111011111111	542EFF
3	1336159	101000110001101011111	14635F
4	6750185	11001101111111111101001	66FFE9
5	1771937	110110000100110100001	1B09A1
6	360952	1011000000111111000	581F8
7	5672561	10101101000111001110001	568E71
8	916583	11011111110001100111	DFC67
9	2700138	1010010011001101101010	29336A
10	9645053	100100110010101111111101	932BFD
11	1181110	100100000010110110110	1205B6
12	1492185	101101100010011011001	16C4D9
13	4018595	1111010101000110100011	3D51A3
14	7654888	11101001100110111101000	74CDE8
15	7062453	11010111100001110110101	6BC3B5
16	2478010	1001011100111110111010	25CFBA
17	6134768	10111011001101111110000	5D9BF0
18	8420417	100000000111110001000001	807C41
19	2917489	1011001000010001110001	2C8471
20	3340773	1100101111100111100101	32F9E5
21	1115956	100010000011100110100	110734
22	9172192	1000

#4.2 Ejercicios de programación 1 Ejercicio 3 - Count Words


In [11]:
import os
import sys
from google.colab import drive

# 1. Montar Drive
if not os.path.ismount('/content/drive'):
    drive.mount('/content/drive')

# 2. Navegar a la carpeta P3 (Localizada en mi Tec google drive)
ruta_p3 = "/content/drive/MyDrive/PruebasSoftware2026/A4.2_ArchivosApoyo/P3"

if os.path.exists(ruta_p3):
    os.chdir(ruta_p3)
    print(f"Directorio de trabajo: {os.getcwd()}")
    print("Archivos disponibles:", os.listdir())
else:
    print(f"La ruta {ruta_p3} no existe. Verificar.")

Directorio de trabajo: /content/drive/MyDrive/PruebasSoftware2026/A4.2_ArchivosApoyo/P3
Archivos disponibles: ['TC3.txt', 'TC2.txt', 'TC4.txt', 'TC5.txt', 'TC1.txt', 'wordCount.py', 'TC1.Results.txt', 'TC2.Results.txt', 'TC3.Results.txt', 'TC4.Results.txt', 'TC5.Results.txt']


## Programa Word Count
Programa que realiza un conteo de palabras contenidas en los archivos individuales de tipo .txt

In [12]:
%%writefile wordCount.py
"""
Programa: wordCount.py
Descripción: Cuenta frecuencias de palabras y genera archivos de resultados individuales.
Salida: Archivos [Nombre].Results.txt con formato tabular según lo especificado en las instrucciones
"""
import sys
import time
import os


def get_words(filename):
    """
    Lee el archivo y retorna una lista de palabras.
    """
    words = []
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            for line in file:
                clean_line = line.strip()
                if not clean_line:
                    continue
                # Separar por espacios
                line_words = clean_line.split()
                if not line_words:
                    continue
                words.extend(line_words)
    except FileNotFoundError:
        sys.stderr.write(f"Error: El archivo '{filename}' no existe.\n")
        return None
    except UnicodeDecodeError:
        sys.stderr.write(f"Error: El archivo '{filename}' no es texto válido.\n")
        return None
    except OSError as error:
        sys.stderr.write(f"Error leyendo '{filename}': {error}\n")
        return None
    return words


def count_frequencies(words):
    """
    Cuenta la frecuencia de cada palabra.
    Retorna un diccionario {palabra: frecuencia}.
    """
    if not words:
        return {}

    freq_dict = {}
    for word in words:
        # Se cuenta la palabra tal cual aparece
        if word in freq_dict:
            freq_dict[word] += 1
        else:
            freq_dict[word] = 1
    return freq_dict


def write_results_file(filename, frequencies, elapsed_time):
    """
    Escribe el archivo de resultados específico para el archivo de entrada.
    Formato: Row Labels [tab] Count of [Nombre]
    Orden: Frecuencia Descendente.
    """
    # 1. Definir nombre de salida: TC1.txt -> TC1.Results.txt
    base_name = os.path.basename(filename)
    name_no_ext = os.path.splitext(base_name)[0]
    output_filename = f"{name_no_ext}.Results.txt"

    # 2. Ordenar datos: Frecuencia Descendente, luego Alfabético
    sorted_items = sorted(frequencies.items(), key=lambda item: (-item[1], item[0]))

    lines = []
    # Encabezado estilo tabla dinámica
    lines.append(f"Row Labels\tCount of {name_no_ext}")

    for word, count in sorted_items:
        lines.append(f"{word}\t{count}")

    # Pie de página con tiempo
    footer_info = f"\n\nExecution time: {elapsed_time:.4f} seconds"

    full_content = "\n".join(lines) + footer_info

    # 3. Escribir archivo
    try:
        with open(output_filename, "w", encoding='utf-8') as f:
            f.write(full_content)
        print(f"Generado: {output_filename} ({len(sorted_items)} palabras únicas)")

    except OSError as error:
        sys.stderr.write(f"Error escribiendo '{output_filename}': {error}\n")


def main():
    """Función principal."""
    if len(sys.argv) < 2:
        print("Uso: python wordCount.py TC1.txt TC2.txt ...")
        sys.exit(1)

    input_files = sys.argv[1:]

    # Procesamos cada archivo de forma independiente
    for filename in input_files:
        start_time = time.time()

        words = get_words(filename)

        if words is not None:
            frequencies = count_frequencies(words)

            end_time = time.time()
            elapsed = end_time - start_time

            write_results_file(filename, frequencies, elapsed)
        else:
            print(f"Saltando '{filename}' por errores de lectura.")


if __name__ == "__main__":
    main()

Overwriting wordCount.py


Se aplica el comando `pylint` sobre el archivo *wordCount.py* y se observa que ya no se producen errores. El código esta completo y no presenta problemas.

In [13]:
!pylint wordCount.py

************* Module wordCount
wordCount.py:1:0: C0103: Module name "wordCount" doesn't conform to snake_case naming style (invalid-name)

-----------------------------------
Your code has been rated at 9.85/10



Ejecución del programa con cada uno de los archivos .txt como entrada y resultados en consola.

In [15]:
!python wordCount.py TC1.txt TC2.txt TC3.txt TC4.txt TC5.txt

Generado: TC1.Results.txt (99 palabras únicas)
Generado: TC2.Results.txt (144 palabras únicas)
Generado: TC3.Results.txt (487 palabras únicas)
Generado: TC4.Results.txt (949 palabras únicas)
Generado: TC5.Results.txt (3750 palabras únicas)
